In [ ]:
import ollama
ollama.list()

In [ ]:
system_prompt = """
You are an AI Agent that uses thought, action, action input and observation to solve a problem.
You get a list of tools available to you. You work in a loop where you generate thoughts and actions + action_inputs. You will get observations from your actions.
User the finish action once you are done.

Always uset this json format to generate a repsonse. NEVER WRITE ANYTHING OUTSIDE.
```json
{
    "thought": "<YOUR THOUGHT>",
    "action": "<YOUR ACTION>",
    "action_input": "<YOUR ACTION INPUT>",
}
```

These Tools are available to you:
- web_search
{
    "thought": "The user wants me to search the web to find the president of America",
    "action": "web_search",
    "action_input": "President of a America",
}
- calculator
{
    "thought": "The user wants me to calculate the age of the president plus twenty",
    "action": "calculator",
    "action_input": "1994 + 20",
}
- finish
{
    "thought": "I am done with the task",
    "action": "finish",
    "action_input": "The answer to the questions is ...",
}
"""

def web_search(query: str) -> str:
    if "turtle" in query:
        return "The oldes turtle is 128 years"
    elif "croc" in query:
        return "The oldest croc is 120 years"
    else:
        return "Could not find result, try different query."

def calculator(problem: str) -> str:
    terms = problem.split(" ")
    n1, n2 = int(terms[0]), int(terms[2])
    op = terms[1]
    if op == "+":
        return str(n1 + n2)
    elif op == "-":
        return str(n1 - n2)

def finish(result: str):
    print(result)

actions = {
    web_search.__name__: web_search,
    calculator.__name__: calculator,
    finish.__name__: finish
}

In [25]:
import json

task = 'How old is the oldest turtle minus the oldest corocdile.'
memory = [{"role": "system", "content": system_prompt}, 
          {'role': 'user',  'content': task}]
action = None

while action != "finish":
    response = ollama.chat(model='llama3.1:8b-instruct-q4_0', messages=memory)
    print(response['message']['content'])
    output = json.loads(response['message']['content'])
    action = output["action"]
    observation = actions[action](output["action_input"])
    print(observation)
    model_message = {"role": "assistant", "content": response['message']['content']}
    observation_message = {"role": "user", "content": observation}
    memory.append(model_message)
    memory.append(observation_message)

{
  "thought": "The user wants me to find the age of a turtle and subtract the age of a crocodile.",
  "action": "web_search",
  "action_input": "oldest turtle on record"
}
The oldes turtle is 128 years
{
  "thought": "I now have the age of the oldest turtle, which is 128. I need to find the age of the oldest crocodile and subtract it from the turtle's age.",
  "action": "web_search",
  "action_input": "oldest crocodile on record"
}
The oldest croc is 120 years
{
  "thought": "I have the age of both the oldest turtle (128) and crocodile (120). I can now calculate their difference.",
  "action": "calculator",
  "action_input": "128 - 120"
}
8
{
  "thought": "The result of subtracting the age of the crocodile from the turtle's age is indeed 8. That's the answer to the user's question.",
  "action": "finish",
  "action_input": "The oldest turtle minus the oldest crocodile is 8."
}
The oldest turtle minus the oldest crocodile is 8.
None


In [ ]:
"""
{{ if .System }}<|start_header_id|>system<|end_header_id|>
You are an AI Agent that uses thought, action, action input and observation to solve a problem.
You get a list of tools available to you. You work in a loop where you generate thoughts and actions + action_inputs. You will get observations from your actions.
ONLY generate a response (FINAL RESPONSE) once you are finished.

Always uset this json format to generate a repsonse. NEVER WRITE ANYTHING OUTSIDE.
```json
{
    "thought": "<YOUR THOUGHT>",
    "action": "<YOUR ACTION>",
    "action_input": "<YOUR ACTION INPUT>",
    "response": "<FINAL RESPONSE>"
}
```

These Tools are available to you:
- web_search
{
    "thought": "The user wants me to search the web to find the president of America",
    "action": "web_search",
    "action_input": "President of a America",
    "response": ""
}
- calculator
{
    "thought": "The user wants me to calculate the age of the president plus twenty",
    "action": "calculator",
    "action_input": "1994 + 20",
    "response": ""
}

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>
How old is the oldest turtle minus the oldest corocdile.
{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>
{ "thought": "The user wants me to find the age of the oldest turtle and subtract the age of the oldest crocodile", 
"action": "web_search", 
"action_input": "Oldest turtle species", 
"response": "" }

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>
Observation: The oldest known turtle is estimated to be around 183 million years old.

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>
{ "thought": "The user wants me to find the age of the oldest turtle and subtract the age of the oldest crocodile", 
"action": "web_search", 
"action_input": "Oldest turtle species", 
"response": "" }

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>
Observation: The oldest known turtle is estimated to be around 183 million years old.
{{ .Response }}<|eot_id|>
"""